In [1]:
!which python

/Users/nuthankishoremaddineni/my_env/bin/python


In [22]:
from tensorflow.keras.models import load_model

# Load the pre-saved model
model_path = "chexnet_feature_extractor.h5"
feature_extractor = load_model(model_path)
print("✅ Model loaded successfully!")

import numpy as np
import tensorflow as tf
import cv2

def preprocess_image(image_path):
    """Preprocess image for DenseNet121."""
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img.astype(np.float32) / 255.0  # Normalize
    return img

def extract_features_single(image_path, model):
    """Extract features for a single image using the preloaded model."""
    img = preprocess_image(image_path)  # Shape: (224, 224, 3)
    img = np.expand_dims(img, axis=0)  # Shape: (1, 224, 224, 3)

    # Extract features using the model
    with tf.device('/device:GPU:0'):
        features = model.predict(img, verbose=0)

    return features.squeeze()  # Output shape (1024,)

# Example usage
image_path = "/Users/nuthankishoremaddineni/Desktop/Radilogy_report_generation/testing_Chest_Xray_Images/three.jpg"
features_list = []
feature_vector = extract_features_single(image_path, feature_extractor)
features_list.append(feature_vector)

# Save features
np.save("Features_Of_Test_ChestXray_Image/image_features.npy", feature_vector)
print(f"✅ Features extracted and saved as 'image_features.npy'")


✅ Model loaded successfully!


✅ Features extracted and saved as 'image_features.npy'


## my_envfinetune Environment

In [23]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")  # Use Apple GPU
    print("MPS (Metal GPU) is available.")
else:
    device = torch.device("cpu")  # Fallback to CPU
    print("MPS not available. Running on CPU.")

MPS (Metal GPU) is available.


In [24]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# ✅ Set device (Mac GPU or CPU fallback)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# ✅ Load trained model & tokenizer
model_path = "/Users/nuthankishoremaddineni/Desktop/Radilogy_report_generation/output"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path).to(device)

# ✅ Set model to evaluation mode
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [25]:
import numpy as np
import torch
def generate_radiology_report(features):
    # Convert features to text format
    features_text = ", ".join(map(str, features))
    input_text = f"Generate a detailed radiology report with features: {features_text}."

    # Tokenize input
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

    # Generate output
    with torch.no_grad():
        output_ids = model.generate(**inputs, max_length=256)

    # Decode and return the generated report
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)
# ✅ Load the .npy file
file_path = "/Users/nuthankishoremaddineni/Desktop/Radilogy_report_generation/Features_Of_Test_ChestXray_Image/image_features.npy"  # Update with your actual file path
features_array = np.load(file_path)  # Load the numpy array

# ✅ Convert to list if needed
features_list = features_array.tolist()

# ✅ Generate Report
generated_report = generate_radiology_report(features_list)

# ✅ Print the Generated Report
print("\nGenerated Radiology Report:\n", generated_report)


Generated Radiology Report:
 Indication: XXXX-year-old female with chest pain Findings: The heart size and mediastinum are within normal limits. The lungs are clear. There is no pneumothorax or pleural effusion. There is no pneumothorax. Impression: No acute cardiopulmonary abnormality.
